<a href="https://colab.research.google.com/github/tuhinjubcse/PoetryTranslationEMNLP2021/blob/main/comet_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## set up comet

In [1]:
!pip install unbabel-comet
!pip install torch==1.9.0 tensorboard~=2.6 imgaug==0.2.5

     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 804 kB 10.0 MB/s 
     |████████████████████████████████| 306 kB 43.5 MB/s 
     |████████████████████████████████| 748.8 MB 18 kB/s 
     |████████████████████████████████| 25.9 MB 944 bytes/s 
     |████████████████████████████████| 1.1 MB 49.8 MB/s 
     |████████████████████████████████| 2.8 MB 58.2 MB/s 
     |████████████████████████████████| 269 kB 59.6 MB/s 
     |████████████████████████████████| 90 kB 8.7 MB/s 
     |████████████████████████████████| 22.3 MB 7.9 MB/s 
     |████████████████████████████████| 2.8 MB 52.4 MB/s 
     |████████████████████████████████| 103 kB 55.0 MB/s 
     |████████████████████████████████| 90 kB 8.6 MB/s 
     |████████████████████████████████| 119 kB 65.8 MB/s 
     |████████████████████████████████| 829 kB 45.2 MB/s 
     |████████████████████████████████| 282 kB 66.3 MB/s 
     |████████████████████████████████| 118 kB 49.5 MB/s 
     |███████████

In [2]:
from comet.models import load_checkpoint, download_model
model = download_model("wmt-large-da-estimator-1719", "/content/drive/MyDrive/PoeticTranslation/comet_model/") # insert your path to COMET model here

public-models.yaml: 8.19kB [00:00, 18.9kB/s]

wmt-large-da-estimator-1719 is already in cache.
Download succeeded. Loading model...



xlmr.large.tar.gz: 1.03GB [00:47, 21.5MB/s]                            


loading archive file /root/.cache/torch/unbabel_comet/xlmr.large
| dictionary: 250001 types


## get data

In [3]:
!git clone https://github.com/tuhinjubcse/PoetryTranslationEMNLP2021.git

Cloning into 'PoetryTranslationEMNLP2021'...
remote: Enumerating objects: 1403, done.
remote: Counting objects: 100% (1403/1403), done.
remote: Compressing objects: 100% (1031/1031), done.
remote: Total 1403 (delta 371), reused 1345 (delta 345), pack-reused 0
Receiving objects: 100% (1403/1403), 17.56 MiB | 20.25 MiB/s, done.
Resolving deltas: 100% (371/371), done.


In [7]:
import os
import json

test_data_path = "/content/PoetryTranslationEMNLP2021/testdatawithtranslations"
SYMBOL = "----" # symbol separating different poems
for folder in os.listdir(test_data_path):
  print(folder)

  #load src
  with open(test_data_path+"/"+folder+"/"+"test"+folder+".txt", "r") as f_:
    lines_src = f_.readlines() # will need the src lines to filter out invalid inputs 
    src = [l.strip() for l in lines_src if SYMBOL not in l]
  
  #load gold
  with open(test_data_path+"/"+folder+"/"+"gold"+folder+".txt", "r") as f_:
    ref = [l.strip() for l in f_.readlines() if SYMBOL not in l]
  
  assert len(src) == len(ref)

  # rest of files are model outputs
  model_outputs = [f for f in os.listdir(test_data_path+"/"+folder) \
                   if "gold" not in f and "test" not in f]
  
  for f in model_outputs:
    print(f)
    mt = []
    with open(test_data_path+"/"+folder+"/"+f, "r") as f_:
      lines_mt = f_.readlines()
      for line_mt, line_src in zip(lines_mt, lines_src):
        if SYMBOL not in line_src: # discard translations corresponding to separator line
          mt.append(line_mt.strip())
      try:
        assert len(mt) == len(src)
      except:
        print("EXCEPTION: ", f)

      data = {"src": src, "mt": mt, "ref": ref}
      data = [dict(zip(data, t)) for t in zip(*data.values())]
      output, scores = model.predict(data, cuda=True, show_progress=True)
      corpus_level = sum(scores)/len(scores)

      # replace with your file paths
      
      # this is to save individual file outputs
      # with open("/content/drive/MyDrive/PoeticTranslation/comet_model/outputs/"+\
      #           "cometScore_"+f[:-4]+".jsonl", "w") as out_f:
      #   for out in output:
      #     json.dump(out, out_f, ensure_ascii=False)
      #     out_f.write('\n')
      
      # replace with your file paths
      with open("/content/drive/MyDrive/PoeticTranslation/comet_model/outputs/"+\
                "total_cometScores.txt", "a") as out_f:
        out_f.write(f[:-4]+'\t'+str(corpus_level))
        out_f.write('\n')